# Importing Data from SQL Database

Step 1: Download the sample sqlite database from: http://www.sqlitetutorial.net/sqlite-sample-database/

Step 2: Use sqlite3 package to make a connection to sqlite database from the downloaded file.

Step 3: Make five different SQL queries that use commands like SELECT, WHERE, GROUP BY and JOIN (but don't limit yourself to these 4 only). You are welcome to go with more advanced queries (such as PARTITION BY, JOIN, etc.)

In [1]:
import pandas as pd
import sqlite3 as sql

In [2]:
db = 'chinook.db'

### List all tables in db

In [3]:
conn = sql.connect(db)

def sql_fetch(con): 
    cursorObj = con.cursor()
 
    cursorObj.execute('SELECT name FROM sqlite_master WHERE type= "table"')
 
    print(cursorObj.fetchall())
 
sql_fetch(conn)

#Close the connection
conn.close()

[('albums',), ('sqlite_sequence',), ('artists',), ('customers',), ('employees',), ('genres',), ('invoices',), ('invoice_items',), ('media_types',), ('playlists',), ('playlist_track',), ('tracks',), ('sqlite_stat1',)]


### Queries to be executed

In [4]:
# which customers have their support rep id between 3 and 4?
query0 = """SELECT * 
            FROM customers 
            WHERE SupportRepId 
            BETWEEN '3' AND '4';"""

In [5]:
# which customers are either from Canada or have a yahoo email address?
query1 = """SELECT * 
            FROM customers 
            WHERE Country = 'Canada' or Email like '%yahoo%';"""

In [6]:
# which customers are from the united states? arrange them alphabetically by surname
query2 = """SELECT * 
            FROM customers 
            WHERE Country = 'USA'
            GROUP BY Lastname;"""

In [7]:
# Which London customers are in there? show only address and first names
query3 = """SELECT Address, FirstName 
            FROM customers 
            WHERE city = 'London' ;"""

In [8]:
# Which customers reside in Italy and have null value for company? Only postcode and lastname
query4 = """SELECT PostalCode, LastName 
            FROM customers 
            WHERE Country = 'Italy' AND Company = 'NULL' ;"""

In [9]:
# which customers use gmail email addresses? only last name and addresses required
query5 = """SELECT LastName, Address 
            FROM customers 
            WHERE Email LIKE '%gmail%' ;"""

In [10]:
# Which artists have an Id 100 or below?
query6 = """SELECT * 
            FROM artists
            WHERE ArtistId <= 100;"""

In [11]:
# Display the albums table
query7 = """SELECT * 
            FROM albums;"""

In [12]:
# List all artist Ids and their corresponding album titles from the artists and albums tables
query8 = """SELECT artists.ArtistId AS ArtistID, albums.Title AS Title
            FROM artists
            JOIN albums
            ON artists.ArtistId = albums.ArtistId;"""

In [13]:
# Based on customer Id, which countries have the highest purchase?
query9 = """SELECT DISTINCT BillingCountry,
            SUM(Total) OVER(PARTITION BY CustomerId) AS TotalOrderAmount
            FROM invoices ;"""

### Connect to db and create a cursor for interaction

In [14]:
# start connection
conn = sql.connect(db)

# allows you to execute queries
#cur = conn.cursor()

### Execute queries

In [15]:
#cur.execute(query1)

#loop to run all queries into a dictionary of dataframes
queries = [query0, query1, query2, query3, query4, query5, query6, query7, query8, query9]
df={}

for query in queries:
    df_ = pd.read_sql_query(sql=query, con=conn) 
    df[query] = df_
    

#Close the connection
conn.close()

### Access the dataframe with specific query as key

In [16]:
df[query9].head(5)

,BillingCountry,TotalOrderAmount
0,Brazil,39.62
1,Germany,37.62
2,Canada,39.62
3,Norway,39.62
4,Czech Republic,40.62


In [17]:
df[query8].head()

,ArtistID,Title
0,1,For Those About To Rock We Salute You
1,2,Balls to the Wall
2,2,Restless and Wild
3,1,Let There Be Rock
4,3,Big Ones


In [18]:
df[query1].head()

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
1,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
2,14,Mark,Philips,Telus,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,+1 (780) 434-4554,+1 (780) 434-5565,mphilips12@shaw.ca,5
3,15,Jennifer,Peterson,Rogers Canada,700 W Pender Street,Vancouver,BC,Canada,V6C 1G8,+1 (604) 688-2255,+1 (604) 688-8756,jenniferp@rogers.ca,3
4,23,John,Gordon,None,69 Salem Street,Boston,MA,USA,2113,+1 (617) 522-1333,None,johngordon22@yahoo.com,4
